In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from pymongo import MongoClient
# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import json
from bson import ObjectId
# Define filter options with breed categories DONE 
filter_options = [
    {'label': 'Water Rescue', 'value': 'Labrador Retriever Mix, Chesapeake Bay Retriever, Newfoundland'},
    {'label': 'Mountain or Wilderness Rescue', 'value': 'German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler'},
    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler'},
    {'label': 'Reset', 'value': 'Reset'}
]



#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name DONE
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name DONE 
class AnimalShelter:
    def __init__(self, user, passwd, host, port, database, collection):
        # Assign values to class variables using the provided parameters
        self.USER = 'aacuser2'
        self.PASS = 'apple1'
        self.HOST = 'nv-desktop-services.apporto.com'
        self.PORT = 30153
        self.DB = 'AAC'
        self.COL = 'animals'
        # Use these variables to establish connection to MongoDB
        try:
            self.client = MongoClient(f'mongodb://{self.USER}:{self.PASS}@{self.HOST}:{self.PORT}')
            self.database = self.client[self.DB]
            self.collection = self.database[self.COL]
        except Exception as e:
            print(f"An error occurred while connecting to MongoDB: {str(e)}")

    def read(self, query):
        # Implement the read method to retrieve data from MongoDB
        try:
            result = self.collection.find(query)
            return list(result)
        except Exception as e:
            print(f"An error occurred while reading data from MongoDB: {str(e)}")

# Connect to database via CRUD Module
# Initialize AnimalShelter class with hardcoded connection details
db = AnimalShelter('aacuser2', 'apple1', 'nv-desktop-services.apporto.com', 30153, 'AAC', 'animals')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo DONE 
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design DONE 
#FIX ME Also remember to include a unique identifier such as your name or date DONE 
html.Div([
    html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='unique_identifier'),
        href='https://www.snhu.edu'
    ),
    html.P('Jorge Torres')  # Display the unique identifier as text
])

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Div([
            html.A(
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='unique_identifier'),
                href='https://www.snhu.edu'
            ),
            html.P('Jorge Torres')  # Display the unique identifier as text
        ])
    ),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        dcc.Dropdown(
            id='filter-dropdown',
            options=filter_options,
            value=None,  # Default value
            placeholder='Select an option...',
            clearable=True
        ),
        html.Hr(),
        dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         page_size=5

#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here DONE
                       ),
        html.Br(),
        html.Hr(),
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ]
        )
    ])
])
#############################################
# Interaction Between Components / Controller
#############################################



    

## FIX ME Add code to filter interactive data table with MongoDB queries DONE 
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value')])
def update_dashboard(filter_type):
    print("Selected filter type:", filter_type)  # Print out the selected filter type for debugging
    if filter_type is None or filter_type == 'Reset':
        # If no filter is selected or if 'Reset' is selected, return the original data
        data = df.to_dict('records')
    else:
        # Split the selected breed categories by comma and strip any whitespace
        breeds = [breed.strip() for breed in filter_type.split(',')]

        # Filter data based on selected breed categories
        filtered_data = [record for record in df.to_dict('records') if record['breed'] in breeds]

        data = filtered_data

    return data
# Display the breeds of animal based on quantity represented in
# the data table

    ###FIX ME #### DONE 
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []

    df = pd.DataFrame.from_dict(viewData)

    # Create a pie chart showing the count of each breed
    fig = px.pie(df, names='breed', title='Preferred Breeds')

    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    else:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
app.run_server(debug=True)



Dash app running on http://127.0.0.1:20080/
Selected filter type: None
Selected filter type: None
Selected filter type: Labrador Retriever Mix, Chesapeake Bay Retriever, Newfoundland
Selected filter type: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
Selected filter type: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
Selected filter type: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
